In [1]:
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

In [2]:
import warnings
warnings.filterwarnings('ignore')
from transformers import (
    LlamaForCausalLM,
    pipeline,
    AutoTokenizer,AutoModelForCausalLM,
    LlamaModel,
    LlamaConfig, Qwen2VLForConditionalGeneration
)
import transformers
import json

import torch
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline

In [3]:
model_dir = "/data/Models/llama3_8B_Base"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_dir, truncation=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

base_model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    load_in_8bit=False,
    torch_dtype=torch.float32,
    device_map='auto',
)

Loading checkpoint shards: 100%|████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.30s/it]


In [5]:
# base_model = LlamaForCausalLM.from_pretrained(
#     model_dir,
#     load_in_8bit=False,
#     torch_dtype=torch.float16,
#     device_map='auto',
# )

In [6]:
intermediate_outputs = {}

In [7]:
def hook_func(module, inputs, outputs, layer_idx):
    intermediate_outputs[layer_idx] = outputs[0]

In [8]:
for layer_idx, layer in enumerate(base_model.model.layers):
    layer.register_forward_hook(
        lambda m, inp, out, idx=layer_idx: hook_func(m, inp, out, idx)
    )

In [9]:
question = "Introduce UT Austin to me"

inputs = tokenizer(question, return_tensors='pt').to(base_model.device)

with torch.no_grad():
    response = base_model.generate(inputs.input_ids, max_length=128)
# print(response)
# print(tokenizer.decode(response.cpu()[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [13]:
for layer_idx, tensor in intermediate_outputs.items():
    print(f"Layer {layer_idx}: {tensor.shape}")

Layer 0: torch.Size([1, 1, 4096])
Layer 1: torch.Size([1, 1, 4096])
Layer 2: torch.Size([1, 1, 4096])
Layer 3: torch.Size([1, 1, 4096])
Layer 4: torch.Size([1, 1, 4096])
Layer 5: torch.Size([1, 1, 4096])
Layer 6: torch.Size([1, 1, 4096])
Layer 7: torch.Size([1, 1, 4096])
Layer 8: torch.Size([1, 1, 4096])
Layer 9: torch.Size([1, 1, 4096])
Layer 10: torch.Size([1, 1, 4096])
Layer 11: torch.Size([1, 1, 4096])
Layer 12: torch.Size([1, 1, 4096])
Layer 13: torch.Size([1, 1, 4096])
Layer 14: torch.Size([1, 1, 4096])
Layer 15: torch.Size([1, 1, 4096])
Layer 16: torch.Size([1, 1, 4096])
Layer 17: torch.Size([1, 1, 4096])
Layer 18: torch.Size([1, 1, 4096])
Layer 19: torch.Size([1, 1, 4096])
Layer 20: torch.Size([1, 1, 4096])
Layer 21: torch.Size([1, 1, 4096])
Layer 22: torch.Size([1, 1, 4096])
Layer 23: torch.Size([1, 1, 4096])
Layer 24: torch.Size([1, 1, 4096])
Layer 25: torch.Size([1, 1, 4096])
Layer 26: torch.Size([1, 1, 4096])
Layer 27: torch.Size([1, 1, 4096])
Layer 28: torch.Size([1, 1, 40

In [11]:
# LLM_model = LlamaForCausalLM.from_pretrained(
#     model_dir,
#     load_in_8bit=False,
#     torch_dtype=torch.float16,
#     device_map='auto',
# )

In [12]:
LLM_model = LlamaForCausalLM.from_pretrained(
    model_dir,
    load_in_8bit=False,
    torch_dtype=torch.float16,
    device_map='auto',
)

Loading checkpoint shards: 100%|████████████████████████████████████████████████████| 4/4 [00:03<00:00,  1.30it/s]
